<a href="https://colab.research.google.com/github/LJMfintech/Financial_Data_Analysis_2/blob/main/DataAnalysisHW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload() # FTSE All-World Index 파일 업로드 하기

Saving FTSE All-World Index.csv to FTSE All-World Index.csv


In [ ]:
from google.colab import files
uploaded = files.upload() # dwcountreturns 파일 업로드 하기

Saving dwcountryreturns.csv to dwcountryreturns.csv


In [ ]:
from google.colab import files
uploaded = files.upload() # nsshock 올리기

Saving nsshock.csv to nsshock.csv


In [ ]:
"""HW2 Final Version (Weekend/Holiday Adjusted)."""

import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from datetime import timedelta

# ============================================================
# 0. FTSE All-World → WORLD return 불러오기
# ============================================================

df = pd.read_csv('FTSE All-World Index.csv')

def add_returns_to_df(df, price_col='prccd', return_col='return'):
    df[return_col] = df[price_col].pct_change()
    return df

df_subset = add_returns_to_df(df)
df_returns = df_subset[['datadate', 'return']]
df_returns.dropna(inplace=True)
df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])
df_returns.to_csv('df_returns.csv', index=False)

# ============================================================
# 1. 국가별 수익률 + WORLD merge
# ============================================================

dwcountryreturns = pd.read_csv('dwcountryreturns.csv')
dwcountryreturns['datadate'] = pd.to_datetime(dwcountryreturns['datadate'].astype(str), format='%Y%m%d', errors='coerce')

df_returns = pd.read_csv('df_returns.csv')
df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])

df_merged = pd.merge(df_returns, dwcountryreturns, on='datadate', how='inner')
df_merged_clean = df_merged.dropna()

df_merged_clean = df_merged_clean[['datadate','country','fic','n','currency','return','portret','portretx']]
df_merged_clean.to_csv('df_merged_clean.csv', index=False)

# ============================================================
# 2. NS shock 불러오기
# ============================================================

nsshock = pd.read_csv('nsshock.csv')
nsshock['datadate'] = pd.to_datetime(nsshock['datadate'])

nsshock = nsshock.sort_values("datadate").reset_index(drop=True)
nsshock["next_shock"] = nsshock["datadate"].shift(-1)

print("데이터 불러오기 완료")
print(df_merged_clean.head())
print(nsshock.head())

# ============================================================
# 3. 타임존 lag 정의
# ============================================================

country_time_lag = {
    'AUSTRALIA': 1, 'AUSTRIA': 1, 'BELGIUM': 1, 'BRAZIL': 0,
    'SWITZERLAND': 1, 'CHILE': 0, 'CHINA': 1, 'COLOMBIA': 0,
    'GERMANY': 1, 'DENMARK': 1, 'EGYPT': 1, 'SPAIN': 1,
    'FINLAND': 1, 'FRANCE': 1, 'UNITED KINGDOM': 1,
    'GREECE': 1, 'HONG KONG': 1, 'HUNGARY': 1, 'INDONESIA': 1,
    'INDIA': 1, 'IRELAND': 1, 'ITALY': 1, 'JAPAN': 1,
    'SOUTH KOREA': 1, 'MEXICO': 0, 'MALAYSIA': 1,
    'NETHERLANDS': 1, 'NORWAY': 1, 'NEW ZEALAND': 1,
    'PHILIPPINES': 1, 'POLAND': 1, 'PORTUGAL': 1,
    'SINGAPORE': 1, 'SWEDEN': 1, 'THAILAND': 1,
    'TURKEY': 1, 'TAIWAN': 1, 'SOUTH AFRICA': 1
}

# ============================================================
# 🔥 4. 주말·공휴일 고려된 거래일 추출 함수 (정상 버전)
# ============================================================

def get_next_trading_days(country_df, event_date, window=5):
    """
    shock 이벤트 이후 해당 국가에서 실제 발생한
    다음 window개 거래일을 반환한다.
    (주말·공휴일 자동 제외됨)
    """
    tmp = country_df[country_df['datadate'] >= event_date].sort_values("datadate")
    if len(tmp) < window:
        return None
    return tmp['datadate'].iloc[:window].tolist()


# ============================================================
# 5. 타임존 조정 + 이벤트 창 병합 함수 (최종 정답 버전)
# ============================================================

def merge_shock_with_time_lag(returns_df, shock_df, time_lag_map, event_window_days=5):

    merged_list = []

    for idx, shock_row in shock_df.iterrows():

        announcement = shock_row['datadate']        # shock date
        shock_val    = shock_row['NS']             # NS shock 값

        # -------------------------------------------------------
        # 이 부분이 핵심: next_shock cutoff 사용하지 않음
        # -------------------------------------------------------
        # → shock_i 이후 해당 국가의 실제 거래일만 기준으로 event window 생성
        # -------------------------------------------------------

        for country, lag in time_lag_map.items():

            # 특정 국가 데이터 가져오기
            country_data = returns_df[returns_df['country'] == country].copy()
            if len(country_data) == 0:
                continue

            # shock보다 국가 데이터가 늦게 시작되면 이벤트 불가능 → skip
            if country_data['datadate'].min() > announcement:
                continue

            # event_day 0 기준일 (타임존 lag 반영)
            event_date = announcement + timedelta(days=lag)

            # 🔥 next_shock 은 더 이상 사용하지 않음
            #    대신 해당 국가의 실제 거래일 기반 window 사용
            trading_days = get_next_trading_days(country_data, event_date, event_window_days)

            # 거래일 부족하면 skip
            if trading_days is None:
                continue

            # event_day 0 ~ 4 생성
            for event_day, tdate in enumerate(trading_days):
                row = country_data[country_data['datadate'] == tdate].iloc[0].copy()
                row["NS_shock"] = shock_val
                row["announcement_date"] = announcement
                row["event_day"] = event_day
                merged_list.append(row)

    return pd.DataFrame(merged_list)


# ============================================================
# 6. Problem 1 실행 (정상 작동)
# ============================================================

df_final = merge_shock_with_time_lag(df_merged_clean, nsshock, country_time_lag, event_window_days=5)

df_final = df_final.sort_values(['announcement_date','country','event_day']).reset_index(drop=True)
df_final.to_csv("problem1_final_merged_data.csv", index=False)

print("✓ Problem 1 완료")
print(df_final.head())

/tmp/ipython-input-3665022608.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_returns.dropna(inplace=True)
/tmp/ipython-input-3665022608.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_returns['datadate'] = pd.to_datetime(df_returns['datadate'])


데이터 불러오기 완료
    datadate    country  fic    n currency   return   portret  portretx
0 1986-07-01  AUSTRALIA  AUS   54      AUD  0.00549 -0.001796 -0.001796
1 1986-07-01    DENMARK  DNK   14      DKK  0.00549  0.012794  0.012794
2 1986-07-01  HONG KONG  HKG   16      HKD  0.00549 -0.000580 -0.000580
3 1986-07-01      JAPAN  JPN  638      JPY  0.00549 -0.003055 -0.003055
4 1986-07-01  SINGAPORE  SGP   17      SGD  0.00549  0.009316  0.009316
    datadate  GSS_target  GSS_path      NS next_shock
0 1995-02-01      0.0597    0.0268  0.0419 1995-03-28
1 1995-03-28      0.0258    0.0568  0.0288 1995-05-23
2 1995-05-23      0.0040   -0.0064  0.0009 1995-07-06
3 1995-07-06     -0.1033   -0.3785 -0.1510 1995-08-22
4 1995-08-22      0.0508    0.0217  0.0354 1995-09-26
✓ Problem 1 완료
    datadate    country  fic    n currency    return   portret  portretx  \
0 1995-02-02  AUSTRALIA  AUS  192      AUD  0.000942  0.010802  0.010802   
1 1995-02-03  AUSTRALIA  AUS  192      AUD  0.005058 -0.003422 -0

In [ ]:
# ============================================================
# 7. Problem 2: Market-model AR & CAR (이벤트 이전 기간으로 α, β 추정)
# ============================================================

import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 7-1. 날짜형으로 정리
df_merged_clean['datadate'] = pd.to_datetime(df_merged_clean['datadate'])
df_final['datadate'] = pd.to_datetime(df_final['datadate'])
df_final['announcement_date'] = pd.to_datetime(df_final['announcement_date'])

# ------------------------------------------------------------
# 7-2. 각 (국가, 발표일)별 이벤트 day 0 날짜 정리
#     → 이 날짜를 기준으로 그 이전 구간에서 α, β 추정
# ------------------------------------------------------------
events0 = (
    df_final[df_final['event_day'] == 0]
    [['country', 'announcement_date', 'datadate']]
    .drop_duplicates()
    .rename(columns={'datadate': 'event_date0'})
)

# estimation window 길이와 gap 설정
EST_WINDOW = 250   # 이벤트 이전 250거래일
GAP_DAYS   = 10    # 이벤트 이전 10일은 제외 (–10까지 buffer)

alpha_dict = {}
beta_dict  = {}

for _, ev in events0.iterrows():
    c   = ev['country']
    ann = ev['announcement_date']
    ev0 = ev['event_date0']

    # 해당 국가 전체 일별 데이터
    cdata = df_merged_clean[df_merged_clean['country'] == c].copy()
    cdata = cdata.sort_values('datadate')

    # 🔥 estimation window: 이벤트 day0 기준 (ev0 - GAP_DAYS) 이전 구간 중,
    #    뒤에서부터 250개 거래일만 사용
    cutoff_date = ev0 - pd.Timedelta(days=GAP_DAYS)
    est_sample = cdata[cdata['datadate'] < cutoff_date]

    if len(est_sample) < EST_WINDOW:
        # 데이터가 너무 적으면 이 이벤트는 사용하지 않음
        continue

    est_sample = est_sample.iloc[-EST_WINDOW:]  # 마지막 250개만 사용

    # 시장모형: portret = α + β * return
    X = sm.add_constant(est_sample['return'])
    y = est_sample['portret']
    res = sm.OLS(y, X).fit()

    alpha_dict[(c, ann)] = res.params['const']
    beta_dict[(c, ann)]  = res.params['return']

print(f"추정된 (country, announcement)별 α, β 개수: {len(alpha_dict)}")

# ------------------------------------------------------------
# 7-3. 이벤트 윈도우 관측치에 α, β 매핑 후 AR, CAR 계산
# ------------------------------------------------------------

df_event = df_final.copy()

keys = list(zip(df_event['country'], df_event['announcement_date']))
df_event['alpha'] = [alpha_dict.get(k, np.nan) for k in keys]
df_event['beta']  = [beta_dict.get(k, np.nan) for k in keys]

# α, β가 없는 이벤트(데이터 부족)는 제거
df_event = df_event.dropna(subset=['alpha', 'beta']).copy()

# 예상수익률(E[R])과 AR 계산
df_event['expected_ret'] = df_event['alpha'] + df_event['beta'] * df_event['return']
df_event['AR'] = df_event['portret'] - df_event['expected_ret']

# CAR: 이벤트별 누적 AR
df_event = df_event.sort_values(['country', 'announcement_date', 'event_day'])
df_event['CAR'] = df_event.groupby(['country', 'announcement_date'])['AR'].cumsum()

# Problem 2용 결과 저장
df_event.to_csv("problem2_eventstudy_fixed.csv", index=False)
print("✓ Problem 2 완료 (AR, CAR 계산 끝)")
print(df_event.head())



추정된 (country, announcement)별 α, β 개수: 7930
✓ Problem 2 완료 (AR, CAR 계산 끝)
    datadate    country  fic    n currency    return   portret  portretx  \
0 1995-02-02  AUSTRALIA  AUS  192      AUD  0.000942  0.010802  0.010802   
1 1995-02-03  AUSTRALIA  AUS  192      AUD  0.005058 -0.003422 -0.003576   
2 1995-02-06  AUSTRALIA  AUS  192      AUD  0.006964  0.006739  0.006739   
3 1995-02-07  AUSTRALIA  AUS  192      AUD -0.002499 -0.008094 -0.008094   
4 1995-02-08  AUSTRALIA  AUS  192      AUD -0.003554 -0.013777 -0.013777   

   NS_shock announcement_date  event_day     alpha      beta  expected_ret  \
0    0.0419        1995-02-01          0 -0.000536  0.775722      0.000194   
1    0.0419        1995-02-01          1 -0.000536  0.775722      0.003387   
2    0.0419        1995-02-01          2 -0.000536  0.775722      0.004865   
3    0.0419        1995-02-01          3 -0.000536  0.775722     -0.002475   
4    0.0419        1995-02-01          4 -0.000536  0.775722     -0.003293   

 

In [ ]:
# ============================================================
# 8. Problem 3: Regression (식 (3)과 일치)
# ============================================================

# 이벤트 창의 마지막 날(event_day = 4)의 CAR을 사용
df_reg = df_event[df_event['event_day'] == 4].copy()

model = smf.ols(
    'CAR ~ NS_shock + C(country) + C(announcement_date)',
    data=df_reg
)
result = model.fit()

print(result.summary())

# 과제용 regression table 만들기
reg_table = pd.DataFrame({
    "Variable":   result.params.index,
    "Coefficient": result.params.values,
    "Std.Err":     result.bse.values,
    "t-value":     result.tvalues.values,
    "p-value":     result.pvalues.values
})

reg_table.to_csv("Table2_RegressionResults.csv", index=False)
print("✓ Problem 3 완료 (회귀 결과 Table2 저장)")
print(reg_table.tail())


                            OLS Regression Results                            
Dep. Variable:                    CAR   R-squared:                       0.166
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     5.660
Date:                Sat, 22 Nov 2025   Prob (F-statistic):          9.82e-155
Time:                        13:00:23   Log-Likelihood:                 18821.
No. Observations:                7930   AIC:                        -3.710e+04
Df Residuals:                    7659   BIC:                        -3.521e+04
Df Model:                         270                                         
Covariance Type:            nonrobust                                         
                                                               coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------